In [2]:
!pip install pytorch_pretrained_bert 

    100% |████████████████████████████████| 133kB 7.6MB/s 
    100% |████████████████████████████████| 655kB 22.0MB/s 
    100% |████████████████████████████████| 133kB 19.8MB/s 
    100% |████████████████████████████████| 5.7MB 7.0MB/s 
    100% |████████████████████████████████| 71kB 24.6MB/s 
    100% |████████████████████████████████| 552kB 26.8MB/s 
  Running setup.py bdist_wheel for regex ... done
  Stored in directory: /root/.cache/pip/wheels/90/04/07/b5010fb816721eb3d6dd64ed5cc8111ca23f97fdab8619b5be
Successfully built regex
menpo 0.8.1 has requirement matplotlib<2.0,>=1.4, but you'll have matplotlib 3.0.2 which is incompatible.
menpo 0.8.1 has requirement pillow<5.0,>=3.0, but you'll have pillow 5.4.0 which is incompatible.
menpo 0.8.1 has requirement scipy<1.0,>=0.16, but you'll have scipy 1.2.0 which is incompatible.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [43]:
import sys
import numpy as np
import pandas as pd
import random as rn
import torch
from pytorch_pretrained_bert import BertModel
from torch import nn
#from torchnlp.datasets import imdb_dataset
from pytorch_pretrained_bert import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output
from collections import Counter
from sklearn import model_selection
from sklearn.metrics import classification_report

In [5]:
rn.seed(321)
np.random.seed(321)
torch.manual_seed(321)
torch.cuda.manual_seed(321)

## Prepare the Data

In [6]:
#use this cell when working online
path = '/floyd/home/ed-triage'
data_path = '/floyd/home/data'

In [3]:
#use this cell when working from home
path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/ed-triage'
data_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/data/ED triage project'

In [7]:
df1 = pd.read_csv(data_path +'/targets.csv', index_col=0)
df2 = pd.read_csv(data_path + '/subj_data.csv', index_col=0)
data = pd.concat([df1,df2],axis = 1)

In [8]:
data.head()

,outcome,target,service,target2,discharge,target3,dispo,target4,SubjectiveNotes,MedicalHistory,pmhx,combo,combo_clean
0,discharge,1,discharge,1,discharge,1,NaN,NaN,NaN,NaN,NaN,nan,NaN
1,discharge,1,discharge,1,discharge,1,NaN,NaN,NaN,NaN,NaN,nan,NaN
2,discharge,1,discharge,1,discharge,1,NaN,NaN,NaN,NaN,NaN,nan,NaN
3,discharge,1,discharge,1,discharge,1,NaN,NaN,NaN,NaN,NaN,nan,NaN
4,discharge,1,discharge,1,discharge,1,NaN,NaN,NaN,NaN,NaN,nan,NaN


In [9]:
#lets just play with a limited dataset to see if we can make this work
data = data[['SubjectiveNotes', 'target3']]

In [10]:
data.head()

,SubjectiveNotes,target3
0,NaN,1
1,NaN,1
2,NaN,1
3,NaN,1
4,NaN,1


In [11]:
data.dropna(inplace = True)
len(data)

110457

In [12]:
#I want a tiny dataset to play with at first
data = data.sample(n=1000,random_state=42)

In [13]:
texts = list(data.SubjectiveNotes)
labels = list(data.target3)
len(texts), len(labels)

(1000, 1000)

In [14]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(texts, labels, test_size=0.1, random_state=42)

In [15]:
Counter(y_train), Counter(y_test)

(Counter({0: 93, 1: 807}), Counter({0: 6, 1: 94}))

In [16]:
train_texts = X_train
train_labels = y_train
test_texts = X_test
test_labels = y_test

In [17]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 1004687.02B/s]


In [18]:
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], test_texts))

len(train_tokens), len(test_tokens)                   
                   

(900, 100)

In [19]:
train_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, train_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, test_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")

train_tokens_ids.shape, test_tokens_ids.shape

((900, 512), (100, 512))

In [20]:
train_y = np.array(train_labels)
test_y = np.array(test_labels)
train_y.shape, test_y.shape, np.mean(train_y), np.mean(test_y)

((900,), (100,), 0.8966666666666666, 0.94)

In [21]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]

# Bert Model

In [22]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')

        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba
        

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [24]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'0.0M'

In [25]:
bert_clf = BertBinaryClassifier()
bert_clf = bert_clf.cuda()


100%|██████████| 407873900/407873900 [00:19<00:00, 20423361.51B/s]


In [26]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'438.016512M'

In [27]:
x = torch.tensor(train_tokens_ids[:3]).to(device)
y, pooled = bert_clf.bert(x, output_all_encoded_layers=False)
x.shape, y.shape, pooled.shape

(torch.Size([3, 512]), torch.Size([3, 512, 768]), torch.Size([3, 768]))

In [28]:
y = bert_clf(x)
y.cpu().detach().numpy()

array([[0.48024282],
       [0.3904461 ],
       [0.39035743]], dtype=float32)

In [29]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'8010.245632M'

In [30]:
y, x, pooled = None, None, None
torch.cuda.empty_cache()
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'438.881792M'

# Fine-tune BERT

In [31]:
BATCH_SIZE = 4
EPOCHS = 2

In [73]:
train_y = train_y.astype(float)
test_y = test_y.astype(float)

In [32]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()

test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'438.881792M'

In [33]:
train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)


In [34]:
param_optimizer = list(bert_clf.sigmoid.named_parameters()) 
optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

In [35]:
optimizer = Adam(bert_clf.parameters(), lr=3e-6)

In [36]:
 torch.cuda.empty_cache()

In [38]:
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
        print(str(torch.cuda.memory_allocated(device)/1000000 ) + 'M')
        logits = bert_clf(token_ids, masks)
        
        loss_func = nn.BCELoss()

        batch_loss = loss_func(logits, labels)
        train_loss += batch_loss.item()
        
        
        bert_clf.zero_grad()
        batch_loss.backward()
        

        clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=1.0)
        optimizer.step()
        
        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_dataset) / BATCH_SIZE, train_loss / (step_num + 1)))
        

Epoch:  2
224/225.0 loss: 0.38477824052174886 


In [39]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)

        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])
    

In [40]:
np.mean(bert_predicted)

1.0

In [44]:
print(classification_report(test_y, bert_predicted))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.94      1.00      0.97        94

   micro avg       0.94      0.94      0.94       100
   macro avg       0.47      0.50      0.48       100
weighted avg       0.88      0.94      0.91       100



/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [47]:
class BertTwoInputBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertTwoInputBinaryClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.bert2 = BertModel.from_pretrained('bert-base-uncased')

        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768*2, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, tokens2, masks=None):
        _, pooled_output1 = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        _, pooled_output2 = self.bert(tokens2, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output1 = self.dropout(pooled_output1)
        dropout_output2 = self.dropout(pooled_output2)
        linear_output = self.linear(dropout_output1 + dropout_output2)
        proba = self.sigmoid(linear_output)
        return proba

In [ ]:
bert_clf2 = BertTwoInputBinaryClassifier()
bert_clf2 = bert_clf2.cuda()